In [ ]:
from __future__ import division

import os
import operator
import numpy as np
import matplotlib.pyplot as plt
import cPickle as pickle

from skimage import transform 
from keras.models import Sequential
from keras.layers.core import Dense, Dropout
from keras.layers.advanced_activations import LeakyReLU
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD, Adam

# share GPU
from keras.backend.tensorflow_backend import set_session
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.49
set_session(tf.Session(config=config))

%matplotlib inline

In [ ]:
def plot_images(images, image_dims, max_col=4, file_name='', to_file=False):
    rows = (len(images)-1)// max_col + 1
    for ix, image in enumerate(images):
        plt.subplot(rows, max_col, ix+1)
        plt.imshow(image.reshape(image_dims), cmap='gray_r', interpolation=None)
    plt.tight_layout()
    plt.show()

def plot_performance(*metrics):
    """
    function plots various performance metrics over time.
    provide multiple tuples in the format (metric-name, metric).
    different metrics (tuples) should be provided as consecutive arguments
    """
    plt.figure()
    for label, metric in metrics:
        epochs = range(len(metric))
        plt.plot(metric, label=label)
    plt.grid()
    plt.legend()
    plt.show()

In [ ]:
# data constants
max_size = (32, 32, 1)
data_files = {'cifar': '../data/cifar.npy',
              'mnist': '../data/mnist.npy',
              'lfw': '../data/lfw.npy'}

# load and transform data
data = np.load(data_files['mnist'])
data = data.astype('float32')

if data.shape[1:3] > max_size:
    data = np.array([transform.resize(image, max_size, preserve_range=True, order=0)] 
                    for image in data)

# shuffle data
np.random.shuffle(data)

# 
data_count = data.shape[0]
data_size = data.shape[1:4] if data.shape[3] > 1 else data.shape[1:3]
data_dim = reduce(operator.mul, data.shape[1:])

print 'Loaded data {}'.format(data.shape)

In [ ]:
# latent space generators
def get_uniform_space(high, low, space_size):
    return lambda batch_size: np.random.uniform(low, high, (batch_size, space_size)).astype('float32')

def get_gaussian_space(mean, var, space_size):
    return lambda batch_size: np.random.normal(mean, var, (batch_size, space_size)).astype('float32')

In [ ]:
def get_mlp_model(input_dim):
    # setup optimizer
    d_opt = SGD(lr=0.01, momentum=0.1)
    g_opt = Adam(lr=0.01)

    leaky_alpha = 0.001
    # setup generator network
    generator = Sequential()
    generator.add(Dense(2048*2, input_dim=input_dim))
    generator.add(LeakyReLU(alpha=leaky_alpha))
    generator.add(Dense(1024*2))
    generator.add(LeakyReLU(alpha=leaky_alpha))
    generator.add(Dense(data_dim, activation='linear'))
    generator.compile(loss='binary_crossentropy', optimizer=g_opt)

    # setup discriminator network
    discriminator = Sequential()
    discriminator.add(Dense(2048, input_dim=data_dim))
    discriminator.add(LeakyReLU(alpha=leaky_alpha))
    discriminator.add(Dense(1024))
    discriminator.add(LeakyReLU(alpha=leaky_alpha))
    discriminator.add(Dense(1, activation='sigmoid'))
    discriminator.compile(loss='binary_crossentropy', optimizer=d_opt)

    # setup combined network
    gen_dis = Sequential()
    gen_dis.add(generator)
    discriminator.trainable = False
    gen_dis.add(discriminator)
    gen_dis.compile(loss='binary_crossentropy', optimizer=g_opt)
    return (generator, discriminator, gen_dis)

In [ ]:
# setup experiment
max_epochs = 250
batch_size = 64
latent_space_size = data_dim
z_space = get_uniform_space(0, 1, latent_space_size)
generator, discriminator, gen_dis = get_mlp_model(latent_space_size)

# prepare data
gen_labels = np.ones(2*batch_size)
gen_labels[:batch_size] = 0
gen_disc_labels = np.ones(batch_size)

g_losses = []
d_losses = []
for i in range(max_epochs):
    z_samples = z_space(batch_size).astype('float32')
    n = np.random.randint(0, data_count-batch_size-1)
    x_samples = data[n: n+batch_size].reshape(batch_size, -1)
    
    g_hist = gen_dis.fit(z_samples, gen_disc_labels, epochs=1, verbose=0)
    d_hist = discriminator.fit(np.vstack([generator.predict(z_samples), x_samples]),
                               gen_labels, epochs=1, verbose=0)
    
    g_losses.append(g_hist.history['loss'])
    d_losses.append(d_hist.history['loss'])
    
    if i % 5 == 0:
        print 'G:', g_hist.history['loss']
        print 'D:', d_hist.history['loss']
        z_samples = z_space(batch_size).astype('float32')
        fakes = generator.predict(z_samples[:8,:])
        plot_images(fakes, data_size)

In [ ]:
plot_performance(('g_loss', g_losses), ('d_loss', d_losses))

In [ ]:
data[0].reshape(-1).reshape(data_size).shape